In [1]:
import sys
import os

# Add the upstream directory to sys.path
upstream_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if upstream_dir not in sys.path:
    sys.path.insert(0, upstream_dir)

# Now you can import the module
from opentrons_api import ot2_api
from microtissue_manipulator import core, utils
import numpy as np 
import cv2
import time
import json
import keyboard
# from pynput import keyboard
from configs import paths
import matplotlib.pyplot as plt

In [2]:
utils.create_configuration_profile('standardDeck')

Profile 'standardDeck' already exists at C:\Users\ivand\Desktop\ot2-microtissue-manipulator\configs\standardDeck.


In [2]:
cap = core.Camera()

Using camera without buffer ...
Camera initialized ...


In [3]:
openapi = ot2_api.OpentronsAPI()

In [4]:
openapi.toggle_lights()

<Response [200]>

In [5]:
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


<Response [200]>

In [6]:
openapi.create_run(verbose=True)

Request status:
<Response [201]>
{
  "data": {
    "id": "721289ae-acf2-49d0-a3c5-aa8704077528",
    "ok": true,
    "createdAt": "2024-11-01T23:49:56.924892+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "hasEverEnteredErrorRecovery": false,
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": []
  }
}


<Response [201]>

In [7]:
_ = openapi.get_run_info()

Total number of runs: 20
Current run ID: 721289ae-acf2-49d0-a3c5-aa8704077528
Current run status: idle


In [8]:
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "da6da9e5-84ca-42d6-9c00-766ab9056197",
    "createdAt": "2024-11-01T23:50:07.306471+00:00",
    "commandType": "loadPipette",
    "key": "da6da9e5-84ca-42d6-9c00-766ab9056197",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "8f198455-b39a-484e-805b-ac3b83173b0c"
    },
    "startedAt": "2024-11-01T23:50:07.308431+00:00",
    "completedAt": "2024-11-01T23:50:09.436091+00:00",
    "intent": "setup",
    "notes": []
  }
}


<Response [201]>

In [8]:
_ = openapi.get_position()

{'x': 127.37760524236512, 'y': 0.24289671842729632, 'z': 152.09999999999997}


In [9]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "opentrons_96_tiprack_300ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 9)

Labware ID:
6fb0490f-4f51-4e76-8101-08075f0b96bf



<Response [201]>

In [9]:
custom_labware_path = os.path.join(paths.BASE_DIR,'protocols','vwr_96_tiprack_200ul.json')
with open(custom_labware_path, 'r') as json_file:
    custom_labware = json.load(json_file)

command_dict = {
            "data": custom_labware
        }

command_payload = json.dumps(command_dict)
url = openapi.runs_url + f'/{openapi.run_id}/' + 'labware_definitions'
r = openapi.post(url = url, headers = openapi.HEADERS,
            params={"waitUntilComplete": True}, data = command_payload)

In [10]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "vwr_96_tiprack_200ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 9, namespace='custom_beta',verbose=True)

Labware ID:
756d1bdd-9303-42f1-b2ee-3772eef3fddf



<Response [201]>

In [42]:
command_dict = {
            "data": {
                "commandType": "moveLabware",
                "params": {
                    "labwareId": openapi.labware_dct['9'],
                    "newLocation": "offDeck",
                    "strategy": "manualMoveWithoutPause"
                },
                "intent": "setup"
            }
        }

command_payload = json.dumps(command_dict)
r = openapi.post(url = openapi.commands_url, headers = openapi.HEADERS,
            params={"waitUntilComplete": True}, data = command_payload)

In [43]:
json.loads(r.text)

{'data': {'id': '29acc534-6780-4d36-b3ea-a5d12b586c22',
  'createdAt': '2024-10-28T23:00:47.268940+00:00',
  'commandType': 'moveLabware',
  'key': '29acc534-6780-4d36-b3ea-a5d12b586c22',
  'status': 'succeeded',
  'params': {'labwareId': '910f9cd5-1336-46af-be0d-b405312a395f',
   'newLocation': 'offDeck',
   'strategy': 'manualMoveWithoutPause'},
  'result': {},
  'startedAt': '2024-10-28T23:00:47.270943+00:00',
  'completedAt': '2024-10-28T23:00:47.272947+00:00',
  'intent': 'setup',
  'notes': []}}

In [17]:
r = openapi.pick_up_tip(openapi.labware_dct['9'], "H12")

In [12]:
openapi.labware_dct

{'1': None,
 '2': None,
 '3': None,
 '4': None,
 '5': None,
 '6': None,
 '7': None,
 '8': None,
 '9': 'a79475eb-b7c9-4c10-9df8-286d7bd56da4',
 '10': None,
 '11': None}

In [38]:
json.loads(r.text)

{'data': {'id': '6d0ea145-1de0-4909-83e0-58457a0e1397',
  'createdAt': '2024-10-24T03:39:37.516312+00:00',
  'commandType': 'pickUpTip',
  'key': '6d0ea145-1de0-4909-83e0-58457a0e1397',
  'status': 'failed',
  'params': {'labwareId': '9fb245c4-eb0e-4ff1-ae11-10440de8bc4c',
   'wellName': 'A1',
   'wellLocation': {'origin': 'top', 'offset': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
   'pipetteId': '08b19b0b-6d8d-4652-ba68-d7a21ebf7819'},
  'error': {'id': '41bf86cc-f69b-4a1a-9489-88959da8edab',
   'createdAt': '2024-10-24T03:39:37.524690+00:00',
   'isDefined': False,
   'errorType': 'LabwareNotLoadedError',
   'errorCode': '4000',
   'detail': 'Labware 9fb245c4-eb0e-4ff1-ae11-10440de8bc4c not found.',
   'errorInfo': {},
   'wrappedErrors': []},
  'startedAt': '2024-10-24T03:39:37.518471+00:00',
  'completedAt': '2024-10-24T03:39:37.524690+00:00',
  'intent': 'setup',
  'notes': []}}

In [18]:
openapi.drop_tip(openapi.labware_dct['9'], "H12")

<Response [201]>

In [29]:
openapi.move_relative(axis="z", distance=1)

<Response [201]>

## Move around with camera

In [11]:
def on_mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Clicked at pixel coordinate: ({}, {})".format(x, y))


# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
while True:
    frame = cap.get_frame(undist=False)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        del manual_movement
        break

cv2.destroyAllWindows()

In [7]:
cv2.imwrite("test.jpg", frame)

True

## Record robot positions and write to calibration.json

In [36]:
calibration_profile = 'standardDeck'

def on_mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Clicked at pixel coordinate: ({}, {})".format(x, y))


# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
while True:
    frame = cap.get_frame(undist=False)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        break

cv2.destroyAllWindows()

utils.check_calibration_config(calibration_profile)
calibration_data = utils.load_calibration_config(calibration_profile)
calibration_data['robot_coords'] = manual_movement.positions
utils.save_calibration_config(calibration_profile, calibration_data)

calibration.json file exists.


## Aruco marker test

In [17]:
squaresX=7
squaresY=5 
squareLength=0.022
markerLength=0.011
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, params)
board = cv2.aruco.CharucoBoard((squaresX, squaresY), squareLength, markerLength, aruco_dict)

manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
while True:
 # Capture frame-by-frame
    frame = cap.get_frame(undist=True)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    # print(marker_corners)

    top_left =  []
    if marker_ids is not None:
        for i in range(len(marker_ids)):
            corner = marker_corners[i][0][0]  # Upper left corner
            top_left.append(corner)
            cv2.circle(frame, tuple(corner.astype(int)), 5, (0, 255, 0), -1)
            # cv2.putText(frame, str(marker_ids[i][0]), (int(corner[0]), int(corner[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, f"({int(corner[0])}, {int(corner[1])})", (int(corner[0]), int(corner[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Our operations on the frame come here
    cv2.imshow(cap.window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        break
 
# When everything done, release the capture
# cap.release_camera()
cv2.destroyAllWindows()

In [18]:
top_left

[array([1786.,  564.], dtype=float32),
 array([738., 581.], dtype=float32),
 array([1792., 1264.], dtype=float32),
 array([ 753., 1278.], dtype=float32)]

## Record pixel coordinates and write to calibration.json

In [12]:
x, y, z = openapi.get_position(verbose=False)[0].values()
x,y,z


(155.0, 49.00000000000001, 100.0)

In [13]:
openapi.move_to_coordinates((155,49,100), min_z_height=1, verbose=False)

<Response [201]>

In [35]:
openapi.move_to_coordinates((155,49,100), min_z_height=1, verbose=False)

calibration_profile = 'standardDeck'
squaresX=7
squaresY=5 
squareLength=0.022
markerLength=0.011
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, params)
board = cv2.aruco.CharucoBoard((squaresX, squaresY), squareLength, markerLength, aruco_dict)


window = cap.get_window()
while True:
 # Capture frame-by-frame
    frame = cap.get_frame(undist=True)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    # print(marker_corners)

    top_left =  []
    if marker_ids is not None:
        for i in range(len(marker_ids)):
            corner = marker_corners[i][0][0]  # Upper left corner
            top_left.append(corner)
            cv2.circle(frame, tuple(corner.astype(int)), 5, (0, 255, 0), -1)
            # cv2.putText(frame, str(marker_ids[i][0]), (int(corner[0]), int(corner[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, f"({int(corner[0])}, {int(corner[1])})", (int(corner[0]), int(corner[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Our operations on the frame come here
    cv2.imshow(cap.window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break
 
# When everything done, release the capture
# cap.release_camera()
cv2.destroyAllWindows()

top_left_array = np.array(top_left)
top_left_array.tolist()
x,y,z = openapi.get_position(verbose=False)[0].values()


utils.check_calibration_config(calibration_profile)
calibration_data = utils.load_calibration_config(calibration_profile)

if len(top_left_array) != 4:
    raise ValueError("Markers not detected. Please try again.")
else:
    calibration_data['camera_coords'] = top_left_array.tolist()
    calibration_data['calib_origin'] = [x,y,z]


utils.save_calibration_config(calibration_profile, calibration_data)


calibration.json file exists.


## Write transformation matrix

In [37]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

camera_coords = calibration_data['camera_coords']
robot_coords = calibration_data['robot_coords']
robot_coords = np.array(robot_coords)[:,:2].tolist()

camera_coords = utils.sort_coordinates(camera_coords)
robot_coords = utils.sort_coordinates(robot_coords, reverse_y=True)

robot_to_camera_coords = {tuple(robot_coord): tuple(camera_coord) for robot_coord, camera_coord in zip(robot_coords, camera_coords)}
tf_mtx = utils.compute_tf_mtx(robot_to_camera_coords)

calibration_data['tf_mtx'] = tf_mtx.tolist()

utils.save_calibration_config(calibration_profile, calibration_data)

## Move and pick

In [55]:
with open(paths.PROFILES_DIR, 'r') as json_file:
    calibration_data = json.load(json_file)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]


def on_mouse_click(event, cX, cY, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Clicked at pixel coordinate: ({}, {})".format(cX, cY))
        X, Y, _ = tf_mtx @ (cX, cY, 1)

        x, y, _ = openapi.get_position(verbose=False).values()
        diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
        X += diff[0]
        Y += diff[1]
        print(f"Robot coords: ({x}, {y})")
        print(f"Clicked on: ({X}, {Y})")
        openapi.move_to_coordinates((X, Y, 10), min_z_height=1)

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False).values()
        # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
        openapi.move_to_coordinates((183,49,100), min_z_height=1, verbose=False)


# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False).values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        break

cv2.destroyAllWindows()

## Record calibration offset

In [38]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])


window = cap.get_window()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X, Y

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)


    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10


    if event == cv2.EVENT_LBUTTONDBLCLK:
        for contour in filtered_contours:
            r=cv2.pointPolygonTest(contour, (x,y), False)
            if r>0:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    X, Y, _ = tf_mtx @ (cX, cY, 1)

                    x, y, _ = openapi.get_position(verbose=False)[0].values()
                    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
                    X += diff[0]# + offset[0]
                    Y += diff[1]# + offset[1]
                    
                    print(f"Robot coords: ({x}, {y})")
                    print(f"Clicked on: ({X}, {Y})")
                    openapi.move_to_coordinates((X, Y, 11), min_z_height=1, verbose=False)
                    openapi.aspirate_in_place(flow_rate = 75, volume = 10)

                    
                else:
                    print("Contour center could not be found")

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
        openapi.move_to_coordinates((155,49,100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(0), int(0))
circle_radius = 300
manual_movement = utils.ManualRobotMovement(openapi)

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.circle(frame, circle_center, circle_radius, (255, 0, 0), 2)
    # Create a mask with the same dimensions as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Draw a filled circle on the mask
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)

    # Convert the masked frame to grayscale
    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to the grayscale image
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # Fill the area outside the circle with black pixels
    # Convert the mask to grayscale
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)


    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter the contours to exclude the outermost
    # filtered_contours = [contour for contour, h in zip(contours, hei[0]) if h[3] == 1]
    # Filter the contours by size
    filtered_contours = [contour for contour in contours if 15 < cv2.contourArea(contour) < 300]
    # Draw the contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)




    cv2.imshow(cap.window_name, frame)
    key_pressed = cv2.waitKey(1)
    if key_pressed == ord('o'):
        x, y, _ = openapi.get_position(verbose=False)[0].values()

        calibration_data['offset'] = [x-X, y-Y]

        utils.save_calibration_config(calibration_profile, calibration_data)

    elif key_pressed == ord('d'):
        openapi.dispense_in_place(flow_rate = 75, volume = 10)

    elif key_pressed == ord('a'):
        openapi.aspirate_in_place(flow_rate = 75, volume = 10)


    elif key_pressed == ord('q'):
        keyboard.unhook_all()
        break

        
cv2.destroyAllWindows()

Robot coords: (155.0, 49.0)
Clicked on: (200.9965939359119, 128.2350148646557)
Robot coords: (155.0, 49.0)
Clicked on: (189.13282209561797, 140.4674209833476)
Robot coords: (155.0, 49.0)
Clicked on: (190.1600433717036, 117.41223780319893)
Robot coords: (155.0, 49.0)
Clicked on: (212.3739589259393, 116.99842729152495)
Robot coords: (155.0, 49.0)
Clicked on: (211.6468236163801, 140.1601128854952)


In [65]:
json.loads(r.text)

{'data': {'id': 'b6305d64-aa5d-4177-a2ab-b3fb663916c9',
  'createdAt': '2024-10-24T02:16:16.379713+00:00',
  'commandType': 'blowOutInPlace',
  'key': 'b6305d64-aa5d-4177-a2ab-b3fb663916c9',
  'status': 'succeeded',
  'params': {'flowRate': 25.0,
   'pipetteId': 'd518c835-ccfd-48ea-a2b3-0f4c2b266520'},
  'result': {},
  'startedAt': '2024-10-24T02:16:16.381774+00:00',
  'completedAt': '2024-10-24T02:16:18.238637+00:00',
  'intent': 'setup',
  'notes': []}}

In [60]:
X,Y

(202.91525716931153, 130.2552449039212)

In [61]:
calibration_data['offset']

[-0.39999999999997726, -0.19999999999998863]

In [35]:
with open(paths.PROFILES_DIR, 'r') as json_file:
    calibration_data = json.load(json_file)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])

In [38]:
offset[1]

np.float64(0.5999999999999659)

In [63]:
openapi.get_position(verbose=False)

({'x': 202.4547281787698, 'y': 130.05610912071882, 'z': 4.0}, <Response [201]>)